In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv("sales_data_training.csv",dtype = float)

In [3]:
data.head()

,critic_rating,is_action,is_exclusive_to_us,is_portable,is_role_playing,is_sequel,is_sports,suitable_for_kids,total_earnings,unit_price
0,3.5,1.0,0.0,1.0,0.0,1.0,0.0,0.0,132717.0,59.99
1,4.5,0.0,0.0,0.0,0.0,1.0,1.0,0.0,83407.0,49.99
2,3.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,62423.0,49.99
3,4.5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,69889.0,39.99
4,4.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,161382.0,59.99


In [4]:
X = data.drop('total_earnings',axis = 1).values
X[:2]

array([[  3.5 ,   1.  ,   0.  ,   1.  ,   0.  ,   1.  ,   0.  ,   0.  ,
         59.99],
       [  4.5 ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,   0.  ,
         49.99]])

In [5]:
Y = data[['total_earnings']].values
Y[:5]

array([[ 132717.],
       [  83407.],
       [  62423.],
       [  69889.],
       [ 161382.]])

In [6]:
test =pd.read_csv("sales_data_test.csv",dtype = float)
X_test = test.drop('total_earnings',axis = 1).values


Y_test = test[['total_earnings']].values

In [7]:
X_scaler = MinMaxScaler(feature_range= (0,1))
Y_scaler = MinMaxScaler(feature_range= (0,1))

In [8]:
X_train = X_scaler.fit_transform(X)
Y_train = Y_scaler.fit_transform(Y)

X_test = X_scaler.transform(X_test)
Y_test = Y_scaler.transform(Y_test)

In [9]:
print("Note: Y values were scaled by multiplying by {:.10f} and adding {:.4f}".format(Y_scaler.scale_[0], Y_scaler.min_[0]))

Note: Y values were scaled by multiplying by 0.0000036968 and adding -0.1159


### Build a neural network

In [10]:
# X- 9 features
# Layer 1 - 50
# Layer 2 = 100
# Layer 3 = 50
# Output Layer

In [11]:
# model params
RUN_NAME = "run 2 with 20 nodes"
learning_rate = 0.001
training_epochs = 100
display_step = 5

# no of inputs and outputs in nn
number_of_inputs = 9
number_of_outputs = 1

# no of neurons in each layer of nn
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

In [12]:
# Define Layers

# Input layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None ,number_of_inputs))

# Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable(name="weights1", shape=[number_of_inputs, layer_1_nodes],
                              initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

# Layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes],
                              initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes],
                              initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

# Output layer
with tf.variable_scope('output'):
    weights = tf.get_variable(name="weights4", shape=[layer_3_nodes, number_of_outputs],
                              initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

In [13]:
# define cost
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))


# define Adam optimizer

with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [14]:
# create summary operation to log the progress of the network
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

In [15]:
with tf.Session() as session:
    
#     try:
#         # Loading from disk
#         saver.restore(session, "logs/trained_model.ckpt")
#     except:
        # If loading doesnt work, initialize the variables when loading from a checkpoint
    session.run(tf.global_variables_initializer())
        
    
    training_writer = tf.summary.FileWriter("./logs/{}/training".format(RUN_NAME), session.graph)
    testing_writer = tf.summary.FileWriter("./logs/{}/testing".format(RUN_NAME), session.graph)

    for epoch in range(training_epochs):

        session.run(optimizer, feed_dict={X: X_train, Y: Y_train})
        
        if epoch % 5 == 0:
            training_cost,training_summary = session.run([cost,summary], feed_dict={X: X_train, Y:Y_train})
            testing_cost,testing_summary = session.run([cost,summary], feed_dict={X: X_test, Y:Y_test})
            
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)

            #print(epoch, training_cost, testing_cost)

    # Training completed!

    # final accuracy scores by running the "cost" operation on the training and test data sets
    final_training_cost = session.run(cost, feed_dict={X: X_train, Y: Y_train})
    final_testing_cost = session.run(cost, feed_dict={X: X_test, Y: Y_test})

    print("Final Training cost: {}".format(final_training_cost))
    print("Final Testing cost: {}".format(final_testing_cost))

    # make predictions 
    Y_predicted_scaled = session.run(prediction, feed_dict={X: X_test})

    # Unscale the data back to it's original units
    Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)

    real_earnings = test['total_earnings'].values[0]
    predicted_earnings = Y_predicted[0][0]

    print("The actual earnings of Game #1 were ${}".format(real_earnings))
    print("Our neural network predicted earnings of ${}".format(predicted_earnings))
    
    #savepath = saver.save(session,"logs/trained_model.ckpt")
    #print("Model save path: {}".format(savepath))

Final Training cost: 0.0010401387698948383
Final Testing cost: 0.0011084085563197732
The actual earnings of Game #1 were $247537.0
Our neural network predicted earnings of $234321.5
